#Reload a fresh Keras model from the saved model

In [1]:
import IPython.display
import librosa
import librosa.display
import pandas as pd
import os
import struct
import glob
import soundfile as sf
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.pyplot import specgram
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
import tensorflow as tf
from keras.callbacks import ModelCheckpoint
from datetime import datetime
from sklearn import metrics
import librosa as lb
import IPython.display as ipd
from tensorflow import keras
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, Flatten, Dense, Dropout, MaxPooling2D, GlobalAveragePooling2D, BatchNormalization
from tqdm import tqdm

seed = 42
tf.random.set_seed(seed)
np.random.seed(seed)

#PREPARE SCREAM DATASET

In [2]:
#DATASET USED
#https://www.kaggle.com/datasets/sanzidaakterarusha/scream-dataset?datasetId=1903423&sortBy=dateRun&tab=collaboration

# Se obtienen las diferentes rutas de los datos, tanto audios como metadata y path para obtener las muestras
file_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio'
screams_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/sounds-screams/good'
urbansound8k = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/metadata/UrbanSound8K.csv')
file_viz = glob.glob('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio/fold1/*')

In [3]:
#pd.set_option('display.max_rows', None)
urbansound8k.head()

,slice_file_name,fsID,start,end,salience,fold,classID,class
0,100032-3-0-0.wav,100032,0.0,0.317551,1,5,3,dog_bark
1,100263-2-0-117.wav,100263,58.5,62.500000,1,5,2,children_playing
2,100263-2-0-121.wav,100263,60.5,64.500000,1,5,2,children_playing
3,100263-2-0-126.wav,100263,63.0,67.000000,1,5,2,children_playing
4,100263-2-0-137.wav,100263,68.5,72.500000,1,5,2,children_playing


In [4]:
#Se organiza dataset para seleccionar la clase gun shot de interes y las otras renombrarse como no scream
urbansound8k.loc[urbansound8k["class"] != " ", "class"] = "non_scream"
#Se cambian los valores de la columna classID para identificar unicamente las dos clases de interes
urbansound8k.loc[urbansound8k["classID"] != 10, "classID"] = 0 #non_event
#se cuenta el numero total de muestras de la clase de interes
totalSamples = 0
lastNameFilesScreams=[]
for file in os.listdir(screams_path):
    if file.endswith('.wav'):
        lastNameFilesScreams.append(file)
        totalSamples+=1
print(totalSamples)
#se toman las muestras aleatorias del tamaño de las muestras de la clase de interes para que quede balanceado
urbansound8k[urbansound8k['class'] == "non_scream"].sample(n=totalSamples).head()
#se toman una las muestras aleatorias del tamaño de las muestras de la clase de interes para que quede balanceado
dfNonScream = urbansound8k[urbansound8k['class'] == "non_scream"].sample(n=totalSamples)
#Se crea dataframe con los datos de los audios de sonidos de interes
dfScream = pd.DataFrame(lastNameFilesScreams, columns=['slice_file_name'])
dfScream['classID'] = 1 #scream
dfScream['class'] = 'scream'
dfScream['fold'] = 0
dfScream
#se unen en un solo dataset las clases de interes con muestras balanceadas
dfCompleteScream = dfNonScream.append(dfScream)
dfCompleteScream.count()[0]
dfCompleteScream[dfCompleteScream['class'] == "scream"].count()[0]
#Se cambia el dataset para unicamente tomar las clases de interés para el trabajo
dfCompleteScream.head()

391


<ipython-input-4-a9de57edfae9>:24: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCompleteScream = dfNonScream.append(dfScream)


,slice_file_name,fsID,start,end,salience,fold,classID,class
2852,161129-4-0-11.wav,161129.0,13.775687,17.775687,1.0,8,0,non_scream
6135,34056-2-0-2.wav,34056.0,1.000000,5.000000,2.0,4,0,non_scream
1023,123685-5-0-8.wav,123685.0,7.845506,11.845506,1.0,7,0,non_scream
2481,157867-8-0-10.wav,157867.0,5.000000,9.000000,1.0,1,0,non_scream
1967,14772-7-9-0.wav,14772.0,31.891222,35.786547,1.0,7,0,non_scream


In [5]:
#Se define funcion para extrar las caracteristicas con la librería librosa, obetiendo los coeficientes ceptrales de frecuencia de Mel
#Se realiza un pading en el tamaño para que concuerden los tamaños de las caracteristicas de entrada al modelo.(?)
def extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL):

    samplerate = 22050
    longitudMaxAudio = 4
    max_pad_len = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios

    try:
      audio, sample_rate = librosa.load(file_name, res_type='soxr_hq')
      mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=Nmfcc, n_fft=Nfft, hop_length=NhopL, win_length=NwinL)
      pad_width = max_pad_len - mfccs.shape[1]
      mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None
    #print(mfccs.shape)
    return mfccs

In [6]:
#Se realiza la extracción de caracteristicas, teniendo en cuenta la clase, si el sonido es de la carpeta agregada de la clase explosions va y busca este sonido en la carpeta requerida

def get_features(Nmfcc, Nfft, NhopL, NwinL):
  features = []
  # Iterate through each sound file and extract the features
  for index, row in dfCompleteScream.iterrows():
      if(row["class"]=="non_scream"):
          file_name = os.path.join(os.path.abspath(file_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
          #file_name = ""
      elif(row["class"]=="scream"):
          file_name = os.path.join(os.path.abspath(screams_path),str(row["slice_file_name"]))
          #file_name = ""
      #print(file_name)
      class_label = row["classID"]
      data = extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL)

      features.append([data, class_label])

  # Convert into a Panda dataframe
  featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
  return featuresdf

In [7]:
def splitFeaturesTrainTest(featuresdf, num_rows, num_columns, num_channels):
  X = np.array(featuresdf.feature.tolist())
  y = np.array(featuresdf.class_label.tolist())

  # Encode the classification labels
  le = LabelEncoder()
  yy = to_categorical(le.fit_transform(y))

  # split the dataset
  from sklearn.model_selection import train_test_split

  x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 3)

  x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
  x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
  num_labels = yy.shape[1]
  return x_train, x_test, y_train, y_test, num_labels

In [8]:


#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5


samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 45
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 5
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1


featuresdfScream = get_features(Nmfcc, Nfft, NhopL, NwinL)
x_trainScream, x_testScream, y_trainScream, y_testScream, num_labelsScream = splitFeaturesTrainTest(featuresdfScream, num_rows, num_columns, num_channels)


#PREPARE GUNSHOT DATASET

In [9]:
# Se obtienen las diferentes rutas de los datos, tanto audios como metadata y path para obtener las muestras
file_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio'
explosions_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/sounds-explosions/explosions'
urbansound8k = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/metadata/UrbanSound8K.csv')
file_viz = glob.glob('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio/fold1/*')

In [10]:
#Se organiza dataset para seleccionar la clase gun shot de interes y las otras renombrarse como no gunshot
urbansound8k.loc[urbansound8k["class"] != "gun_shot", "class"] = "non_gun_shot"
#Se cambian los valores de la columna classID para identificar unicamente las dos clases de interes
urbansound8k.loc[urbansound8k["classID"] != 6, "classID"] = 0  #non_event
urbansound8k.loc[urbansound8k["classID"] == 6, "classID"] = 1  #gun_shot

#se cuenta el numero total de muestras de la clase de interes
totalSamplesGunshot =  urbansound8k.loc[((urbansound8k["class"]=="gun_shot"))].count()[0]
totalSamplesGunshot

#se carga el numero total de muestras de la clase a agregar al dataset - explosions
totalSamplesExplosions = 0
lastNameFilesExplosions=[]
for file in os.listdir(explosions_path):
    if file.endswith('.wav'):
        lastNameFilesExplosions.append(file)
        totalSamplesExplosions+=1
print(totalSamplesExplosions)

#Se crea dataframe con los datos de los audios de sonidos de la clase a agregar - explosions
dfExplosion = pd.DataFrame(lastNameFilesExplosions, columns=['slice_file_name'])
dfExplosion['classID'] = 0 #non_event
dfExplosion['class'] = 'non_gun_shot'
dfExplosion['fold'] = 0

#Agregar en un dataset todos los sonidos de interes con la clase nueva agregada - explosions
print(urbansound8k.count()[0])
urbansound8k = urbansound8k.append(dfExplosion)
urbansound8k.count()[0]

#se toma una muestra aleatoria del tamaño de las muestras de la clase de interes para que quede balanceado
dfNonGunShot = urbansound8k[urbansound8k['class'] == "non_gun_shot"].sample(n=totalSamplesGunshot)
dfGunShot= urbansound8k[urbansound8k['class'] == "gun_shot"]

#se unen en un solo dataset las clases de interes con muestras balanceadas
dfCompleteGunshot = dfNonGunShot.append(dfGunShot)
dfCompleteGunshot.count()[0]

209
8732


<ipython-input-10-5260589046a1>:28: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  urbansound8k = urbansound8k.append(dfExplosion)
<ipython-input-10-5260589046a1>:36: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCompleteGunshot = dfNonGunShot.append(dfGunShot)


748

In [11]:
#Se define funcion para extrar las caracteristicas con la librería librosa, obetiendo los coeficientes ceptrales de frecuencia de Mel
#Se realiza un pading en el tamaño para que concuerden los tamaños de las caracteristicas de entrada al modelo.

def extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL):

    samplerate = 22050
    longitudMaxAudio = 4
    max_pad_len = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios

    try:
      audio, sample_rate = librosa.load(file_name, res_type='soxr_hq')
      mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=Nmfcc, n_fft=Nfft, hop_length=NhopL, win_length=NwinL)
      pad_width = max_pad_len - mfccs.shape[1]
      mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None
    #print(mfccs.shape)
    return mfccs

In [12]:
#Se realiza la extracción de caracteristicas, teniendo en cuenta la clase, si el sonido es de la carpeta agregada de la clase explosions va y busca este sonido en la carpeta requerida

def get_features(Nmfcc, Nfft, NhopL, NwinL):
  features = []
  # Iterate through each sound file and extract the features
  for index, row in dfCompleteGunshot.iterrows():
      if(row["fold"]!=0):
          file_name = os.path.join(os.path.abspath(file_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
          #file_name = ""
      elif(row["fold"]==0):
          file_name = os.path.join(os.path.abspath(explosions_path),str(row["slice_file_name"]))
          #file_name = ""
      #print(file_name)
      class_label = row["classID"]
      data = extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL)

      features.append([data, class_label])

  # Convert into a Panda dataframe
  featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
  return featuresdf

In [13]:
def splitFeaturesTrainTest(featuresdf, num_rows, num_columns, num_channels):
  X = np.array(featuresdf.feature.tolist())
  y = np.array(featuresdf.class_label.tolist())

  # Encode the classification labels
  le = LabelEncoder()
  yy = to_categorical(le.fit_transform(y))

  # split the dataset
  from sklearn.model_selection import train_test_split

  x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 3)

  x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
  x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
  num_labels = yy.shape[1]
  return x_train, x_test, y_train, y_test, num_labels

In [14]:

#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 45
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 5
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1


featuresdfGunshot = get_features(Nmfcc, Nfft, NhopL, NwinL)
x_trainGunshot, x_testGunshot, y_trainGunshot, y_testGunshot, num_labelsGunshot = splitFeaturesTrainTest(featuresdfGunshot, num_rows, num_columns, num_channels)

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3969
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3949
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3667
  warnings.warn(


#PREPARE SIREN DATASET

In [15]:
# Se obtienen las diferentes rutas de los datos, tanto audios como metadata y path para obtener las muestras
file_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio'
urbansound8k = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/metadata/UrbanSound8K.csv')
file_viz = glob.glob('/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/UrbanSounds8k/UrbanSound8K/audio/fold1/*')

In [16]:
#Se organiza dataset para seleccionar la clase de interes y las otras renombrarse como no interes
urbansound8k.loc[urbansound8k["class"] != "siren", "class"] = "non_siren"
#Se cambian los valores de la columna classID para identificar unicamente las dos clases de interes
urbansound8k.loc[urbansound8k["classID"] != 8, "classID"] = 0 #non_event
urbansound8k.loc[urbansound8k["classID"] == 8, "classID"] = 1 #siren

#se cuenta el numero total de muestras de la clase de interes
totalSamplesSiren =  urbansound8k.loc[((urbansound8k["class"]=="siren"))].count()[0]
totalSamplesSiren

#se toma una muestra aleatoria del tamaño de las muestras de la clase de interes para que quede balanceado
dfNonSiren = urbansound8k[urbansound8k['class'] == "non_siren"].sample(n=totalSamplesSiren)
dfSiren = urbansound8k[urbansound8k['class'] == "siren"]

#se unen en un solo dataset las clases de interes con muestras balanceadas
dfCompleteSiren = dfNonSiren.append(dfSiren)
dfCompleteSiren.count()[0]

<ipython-input-16-34a8a78805fd>:16: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  dfCompleteSiren = dfNonSiren.append(dfSiren)


1858

In [17]:
#Se define funcion para extrar las caracteristicas con la librería librosa, obetiendo los coeficientes ceptrales de frecuencia de Mel
#Se realiza un pading en el tamaño para que concuerden los tamaños de las caracteristicas de entrada al modelo.

def extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL):

    samplerate = 22050
    longitudMaxAudio = 4
    max_pad_len = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios

    try:
      audio, sample_rate = librosa.load(file_name, res_type='soxr_hq')
      mfccs = librosa.feature.mfcc(y=audio, sr=sample_rate, n_mfcc=Nmfcc, n_fft=Nfft, hop_length=NhopL, win_length=NwinL)
      pad_width = max_pad_len - mfccs.shape[1]
      mfccs = np.pad(mfccs, pad_width=((0, 0), (0, pad_width)), mode='constant')

    except Exception as e:
      print("Error encountered while parsing file: ", file_name)
      return None
    #print(mfccs.shape)
    return mfccs

In [18]:
#Se realiza la extracción de caracteristicas, teniendo en cuenta la clase, si el sonido es de la carpeta agregada de la clase explosions va y busca este sonido en la carpeta requerida

def get_features(Nmfcc, Nfft, NhopL, NwinL):
  features = []
  # Iterate through each sound file and extract the features
  for index, row in dfCompleteSiren.iterrows():
      if(row["fold"]!=0):
          file_name = os.path.join(os.path.abspath(file_path),'fold'+str(row["fold"])+'/',str(row["slice_file_name"]))
          #file_name = ""
      elif(row["fold"]==0):
          file_name = os.path.join(os.path.abspath(explosions_path),str(row["slice_file_name"]))
          #file_name = ""
      #print(file_name)
      class_label = row["classID"]
      data = extract_features(file_name, Nmfcc, Nfft, NhopL, NwinL)

      features.append([data, class_label])

  # Convert into a Panda dataframe
  featuresdf = pd.DataFrame(features, columns=['feature','class_label'])
  return featuresdf

In [19]:
def splitFeaturesTrainTest(featuresdf, num_rows, num_columns, num_channels):
  X = np.array(featuresdf.feature.tolist())
  y = np.array(featuresdf.class_label.tolist())

  # Encode the classification labels
  le = LabelEncoder()
  yy = to_categorical(le.fit_transform(y))

  # split the dataset
  from sklearn.model_selection import train_test_split

  x_train, x_test, y_train, y_test = train_test_split(X, yy, test_size=0.2, random_state = 3)

  x_train = x_train.reshape(x_train.shape[0], num_rows, num_columns, num_channels)
  x_test = x_test.reshape(x_test.shape[0], num_rows, num_columns, num_channels)
  num_labels = yy.shape[1]
  return x_train, x_test, y_train, y_test, num_labels

In [20]:

#1	3	4096	4096	4096	0	2
#2	12	4096	4096	4096	0	3
#3	27	4096	4096	4096	0	3
#4	45	4096	4096	4096	0	5

samplerate = 22050
longitudMaxAudio = 4
Nmfcc = 45
Nfft = 4096
NwinL = 4096
iterableNhopL = 1.0
NhopL =  4096       #int(iterableNhopL*NwinL)
k_size = 5
num_rows = Nmfcc
num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
num_channels = 1


featuresdfSiren = get_features(Nmfcc, Nfft, NhopL, NwinL)
x_trainSiren, x_testSiren, y_trainSiren, y_testSiren, num_labelsSiren = splitFeaturesTrainTest(featuresdfSiren, num_rows, num_columns, num_channels)

/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3954
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=1323
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3087
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/librosa/core/spectrum.py:257: UserWarning: n_fft=4096 is too large for input signal of length=3949
  warnings.warn(


In [21]:
x_trainSiren.shape

(1486, 45, 22, 1)

In [22]:
num_labelsSiren

2

In [23]:
num_labelsScream

2

In [24]:
num_labelsGunshot

2

#EVALUATING OG MODELS ARQ 4

In [25]:
modelGunshotTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL = tf.keras.models.load_model(modelGunshotTLSave_path)

modelScreamTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL = tf.keras.models.load_model(modelScreamTLSave_path)

modelSirenTLSave_path = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL = tf.keras.models.load_model(modelSirenTLSave_path)
modelSirenTL.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer='adam')


# Evaluating the model on the training and testing set

score = modelScreamTL.evaluate(x_trainScream, y_trainScream, verbose=0)
print("Training Accuracy SCREAM: ", score[1])

score = modelScreamTL.evaluate(x_testScream, y_testScream, verbose=0)
print("Testing Accuracy SCREAM: ", score[1])

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(modelScreamTL.predict(x_testScream),axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report SCREAM: \n\n',classification_report(y_true,y_pred))

score = modelGunshotTL.evaluate(x_trainGunshot, y_trainGunshot, verbose=0)
print("Training Accuracy GUN: ", score[1])

score = modelGunshotTL.evaluate(x_testGunshot, y_testGunshot, verbose=0)
print("Testing Accuracy GUN: ", score[1])

y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(modelGunshotTL.predict(x_testGunshot),axis=1)
print('\nConfusion Matrix GUN:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report GUN: \n\n',classification_report(y_true,y_pred))
print('\n\nClassification Report GUN F1-SCORE: \n\n', float(classification_report(y_true,y_pred).split()[15]))


score = modelSirenTL.evaluate(x_trainSiren, y_trainSiren, verbose=0)
print("Training Accuracy SIREN: ", score[1])

score = modelSirenTL.evaluate(x_testSiren, y_testSiren, verbose=0)
print("Testing Accuracy SIREN: ", score[1])

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(modelSirenTL.predict(x_testSiren),axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report SIREN: \n\n',classification_report(y_true,y_pred))
print('\n\nClassification Report SIREN F1-SCORE: \n\n', float(classification_report(y_true,y_pred).split()[15]))

Training Accuracy SCREAM:  1.0
Testing Accuracy SCREAM:  0.987261176109314
5/5 [==============================] - 0s 14ms/step

Confusion Matrix SCREAM:


[[83  1]
 [ 1 72]]


Classification Report SCREAM: 

               precision    recall  f1-score   support

           0       0.99      0.99      0.99        84
           1       0.99      0.99      0.99        73

    accuracy                           0.99       157
   macro avg       0.99      0.99      0.99       157
weighted avg       0.99      0.99      0.99       157

Training Accuracy GUN:  0.9765886068344116
Testing Accuracy GUN:  0.9866666793823242
5/5 [==============================] - 0s 14ms/step

Confusion Matrix GUN:


[[79  1]
 [ 1 69]]


Classification Report GUN: 

               precision    recall  f1-score   support

           0       0.99      0.99      0.99        80
           1       0.99      0.99      0.99        70

    accuracy                           0.99       150
   macro avg       0.99      0.99

#SELECCIÓN DE FILTROS PARA FUSION BASADO EN DESEMPEÑO ENTRE MODELOS CON PROMEDIO COMO DESEMPATE ARQ 4

In [26]:
import pandas as pd
from scipy.ndimage import rotate
import copy

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model


def getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size):
    inputs = Input(shape=(num_rows, num_columns, num_channels))
    x = Conv2D(filters=16, kernel_size=k_size, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=32, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=64, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=128, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)

    output1 = Dense(num_labels, activation='softmax', name='output1')(x)
    output2 = Dense(num_labels, activation='softmax', name='output2')(x)

    model = Model(inputs=inputs, outputs=[output1, output2])
    return model



def get_layer_with_params(model, layer_index):
    layers_with_params = [layer for layer in model.layers if len(layer.trainable_weights) > 0]

    if layer_index < 0 or layer_index >= len(layers_with_params):
        raise ValueError("Invalid layer index.")

    return layers_with_params[layer_index]

def transfer_layer_weights(layer_from, layer_to):
    if len(layer_from.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0 or ((len(layer_from.trainable_weights)) != (len(layer_to.trainable_weights))):
        return False  # Ambas capas deben tener parámetros entrenables

    weights_from, biases_from = layer_from.get_weights()
    layer_to.set_weights([weights_from, biases_from])
    return True

def transfer_parameters_between_models(model_from, layer_index_from, model_to, layer_index_to):
    layer_from = get_layer_with_params(model_from, layer_index_from)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_layer_weights(layer_from, layer_to):
        print(f"Transferred weights from layer '{layer_from.name}' to layer '{layer_to.name}'.")
    else:
        print(f"Layers '{layer_from.name}' and '{layer_to.name}' don't have trainable parameters.")


def transfer_filters_and_biases_one_by_one(layer_from, layer_to, idx_filter_from, idx_filter_to):
    if len(layer_from.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0:
        return False  # Ambas capas deben tener parámetros entrenables

    filters_from, biases_from = layer_from.get_weights()
    filters_to, biases_to = layer_to.get_weights()

    if filters_from.shape != filters_to.shape or biases_from.shape != biases_to.shape:
        return False  # Las dimensiones de los filtros y sesgos deben coincidir

    filters_to[..., idx_filter_to] = filters_from[..., idx_filter_from]
    #print(filters_to[..., i] - filters_from[..., i])
    biases_to[idx_filter_to] = biases_from[idx_filter_from]  # Copiar el sesgo i de 'layer_from' a 'layer_to'

    layer_to.set_weights([filters_to, biases_to])
    return True


def transfer_filters_and_biases_one_by_one_between_models(model_from, layer_index_from, idx_filter_from, model_to, layer_index_to, idx_filter_to):
    layer_from = get_layer_with_params(model_from, layer_index_from)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_filters_and_biases_one_by_one(layer_from, layer_to, idx_filter_from, idx_filter_to):
        filters_from, biases_from = layer_from.get_weights()
        filters_to, biases_to = layer_to.get_weights()
        print(f"Transferred filters and biases from layer '{layer_from.name}' to layer '{layer_to.name}' one by one.")
    else:
        print(f"Filters and biases in layers '{layer_from.name}' and '{layer_to.name}' couldn't be transferred one by one.")

    return model_to


def transfer_filters_AVG_and_biases_one_by_one(layer_from1, layer_from2, layer_to, idx_filter_from1, idx_filter_from2, idx_filter_to):
    if len(layer_from1.trainable_weights) == 0 or len(layer_from2.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0:
        return False  # Ambas capas deben tener parámetros entrenables

    filters_from1, biases_from1 = layer_from1.get_weights()
    filters_from2, biases_from2 = layer_from2.get_weights()
    filters_to, biases_to = layer_to.get_weights()

    if filters_from1.shape != filters_to.shape or biases_from1.shape != biases_to.shape or filters_from2.shape != filters_to.shape or biases_from2.shape != biases_to.shape:
        return False  # Las dimensiones de los filtros y sesgos deben coincidir

    # Intercambiar los filtros
    filter_from1 = np.copy(filters_from1[:, :, :, idx_filter_from1])
    filter_from2 = np.copy(filters_from2[:, :, :, idx_filter_from2])

    bias_from1 = np.copy(biases_from1[idx_filter_from1])
    bias_from2 = np.copy(biases_from2[idx_filter_from2])

    # Promediar los pesos y bias de los filtros
    avg_filter = (filter_from1 + filter_from2) / 2
    avg_bias = (bias_from1 + bias_from2) / 2

    filters_to[..., idx_filter_to] = avg_filter
    #print(filters_to[..., i] - avg_filter)
    biases_to[idx_filter_to] = avg_bias # Copiar el sesgo i de 'layer_from' a 'layer_to'

    layer_to.set_weights([filters_to, biases_to])
    return True


def transfer_filters_and_biases_one_by_one_AVG_between_models(model_from1, layer_index_from1, idx_filter_from1, model_from2, layer_index_from2, idx_filter_from2, model_to, layer_index_to, idx_filter_to):
    layer_from1 = get_layer_with_params(model_from1, layer_index_from1)
    layer_from2 = get_layer_with_params(model_from2, layer_index_from2)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_filters_AVG_and_biases_one_by_one(layer_from1, layer_from2, layer_to, idx_filter_from1, idx_filter_from2, idx_filter_to):
        filters_from1, biases_from1 = layer_from1.get_weights()
        filters_from2, biases_from2 = layer_from2.get_weights()
        filters_to, biases_to = layer_to.get_weights()
        print(f"Transferred filters and biases from layer '{layer_from1.name}' and '{layer_from2.name}' to layer '{layer_to.name}' one by one.")
    else:
        print(f"Filters and biases in layers '{layer_from1.name}' '{layer_from2.name}' and '{layer_to.name}' couldn't be transferred one by one.")

    return model_to



def interchange_filters_between_models(model1, model2, x_testModel1, y_testModel1, x_testModel2, y_testModel2, fusionDataOutPath, fusedModelSavePath):


    #1	3	4096	4096	4096	0	2
    #2	12	4096	4096	4096	0	3
    #3	27	4096	4096	4096	0	3
    #4	45	4096	4096	4096	0	5

    samplerate = 22050
    longitudMaxAudio = 4
    Nmfcc = 45
    Nfft = 4096
    NwinL = 4096
    iterableNhopL = 1.0
    NhopL =  4096       #int(iterableNhopL*NwinL)
    k_size = 5
    num_rows = Nmfcc
    num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
    num_channels = 1

    num_labels = 2 #4

    #model = getModel(num_rows, num_columns, num_channels, num_labels, k_size)
    #model = getModelFunctional(num_rows, num_columns, num_channels, num_labels, k_size)
    Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

    # Compila el modelo (es necesario antes de la evaluación)
    Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #COPY ORIGINAL MODELS
    model1_copy= keras.models.clone_model(model1)
    model1_copy.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model1_copy.set_weights(model1.get_weights())

    model2_copy= keras.models.clone_model(model2)
    model2_copy.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model2_copy.set_weights(model2.get_weights())


    layer_names = [layer.name for layer in model1.layers if 'conv' in layer.name]

    data = []

    for index_layer in range(len(layer_names)):

      layer_name = layer_names[index_layer]

      fusedLayer = get_layer_with_params(Fusedmodel, index_layer)
      #fusedLayer = Fusedmodel.get_layer(layer_name)
      filtersFused, biasesFused = fusedLayer.get_weights()

      layer1 = get_layer_with_params(model1, index_layer)
      #layer1 = model1.get_layer(layer_name)
      filters1, biases1 = layer1.get_weights()

      layer2 = get_layer_with_params(model2, index_layer)
      #layer2 = model2.get_layer(layer_name)
      filters2, biases2 = layer2.get_weights()

      n_filters1 = filters1.shape[-1]
      n_filters2 = filters2.shape[-1]


      # Interchange the filters
      number_filters = 0

      for filter_idx in range(n_filters1):

        transfer_filters_and_biases_one_by_one_between_models(model1, index_layer, filter_idx, model2, index_layer, filter_idx)

        # Get the filters
        filter1 = filters1[:, :, :, filter_idx]
        filter2 = filters2[:, :, :, filter_idx]

        # Get the filters
        bias1 = biases1[filter_idx]
        bias2 = biases2[filter_idx]

        #PROBAMOS PRIMERO EL PRIMER FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
        model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx, model2, index_layer, filter_idx)
        model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx, model1, index_layer, filter_idx)

        # Evaluating models on the training and testing set
        y_true_model2 = np.argmax(y_testModel2,axis=1)
        y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
        #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
        f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

        y_true_model1 = np.argmax(y_testModel1,axis=1)
        y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
        #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
        f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

        F1_prom1 = (f1_score_model2+f1_score_model1)/2
        print('\n\nF1-SCORE PROM 1: \n', F1_prom1)
        #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
        #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


        #PROBAMOS DESPUES EL SEGUNDO FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
        model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx, model2, index_layer, filter_idx)
        model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx, model1, index_layer, filter_idx)

        # Evaluating models on the training and testing set
        y_true_model2 = np.argmax(y_testModel2,axis=1)
        y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
        #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
        f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

        y_true_model1 = np.argmax(y_testModel1,axis=1)
        y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
        #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
        f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


        F1_prom2 = (f1_score_model2+f1_score_model1)/2
        print('\n\nF1-SCORE PROM 2: \n', F1_prom2)
        #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
        #print('\nF1-SCORE MODEL 2: ', f1_score_model2)



        #VALIDAR CON CUÄL FILTRO SE OBTUVO MEJOR DESEMPEÑO
        if(F1_prom1>F1_prom2):
            #ME QUEDO CON EL FILTRO 1 EN AMBOS MODELOS
            # Update the filters in the models
            model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx, model2, index_layer, filter_idx)
            model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx, model1, index_layer, filter_idx)

            Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx, Fusedmodel, index_layer, filter_idx)

            #filtersFused, biasesFused = fusedLayer.get_weights()
            #filters2, biases2 = layer2.get_weights()
            #print(filtersFused[:, :, :, filter_idx] - filters2[:, :, :, filter_idx])

            Selected = 'Filter1'

        elif(F1_prom1<F1_prom2):
            #ME QUEDO CON EL FILTRO 2 EN AMBOS MODELOS
            # Update the filters in the models
            model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx, model2, index_layer, filter_idx)
            model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx, model1, index_layer, filter_idx)

            Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx, Fusedmodel, index_layer, filter_idx)

            #filtersFused, biasesFused = fusedLayer.get_weights()
            #filters2, biases2 = layer2.get_weights()
            #print(filtersFused[:, :, :, filter_idx] - filters2[:, :, :, filter_idx])

            Selected = 'Filter2'

        else:
            #ME QUEDO CON EL  PROMEDIO ENTRE AMBOS FILTROS DE LOS MODELOS
            # Update the filters in the models
            model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, model2, index_layer, filter_idx)
            model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, model1, index_layer, filter_idx)

            Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, Fusedmodel, index_layer, filter_idx)


            #filtersFused, biasesFused = fusedLayer.get_weights()
            #filters2, biases2 = layer2.get_weights()
            #print(filtersFused[:, :, :, filter_idx] - filters2[:, :, :, filter_idx])

            Selected = 'AVERAGE'


        data.append((index_layer + 1, filter_idx + 1, F1_prom1, F1_prom2, Selected))
        number_filters += 1
      print(f"Filters interchanged successfully for Layer {index_layer} between the models.")

    df = pd.DataFrame(data, columns=['LayerModel', 'FilterModel', 'filter1_f1_AVG', 'filter2_f1_AVG', 'Selected'])
    df.to_excel(fusionDataOutPath, index=False)
    print("File saved correctly.")

    #transfer_parameters_between_models(modelGunshotTL, 0, Fusedmodel, 0)
    #transfer_parameters_between_models(modelGunshotTL, 1, Fusedmodel, 1)
    #transfer_parameters_between_models(modelGunshotTL, 2, Fusedmodel, 2)
    #transfer_parameters_between_models(modelGunshotTL, 3, Fusedmodel, 3)
    transfer_parameters_between_models(model1, 4, Fusedmodel, 4)
    transfer_parameters_between_models(model2, 4, Fusedmodel, 5)

    print(f"Filters interchanged equal to {number_filters}.")
    # Evaluating FINAL PERFORMANCE of models on the training and testing set
    y_true_model2 = np.argmax(y_testModel2,axis=1)
    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
    print('\n\nClassification Report - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

    y_true_model1 = np.argmax(y_testModel1,axis=1)
    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
    print('\n\nClassification Report - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

    Fusedmodel.save(fusedModelSavePath)

    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(fusedModelSavePath) # path to the SavedModel directory
    tflite_model = converter.convert()

    # Save the model.
    with open(fusedModelSavePath+".tflite", 'wb') as f:
      f.write(tflite_model)

    print("Saved Models Correctly.")

    return Fusedmodel

#GUN VS SCREAM OPT AVG ARQ 4

In [27]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


#fusionDataOutPathGunScreamSiren =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream_siren/FusionData/filtersFusion_performance_optAVG.xlsx"

fusionDataOutPathGunScream_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/FusionData/filtersFusion_performance_optAVG_4.xlsx"
fusedModelSavePathGunScream_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/fused_models/fused_gun_scream_optAVG_4"


OutFusedModelGunScream = interchange_filters_between_models(modelGunshotTL_4, modelScreamTL_4,  x_testGunshot, y_testGunshot, x_testScream, y_testScream, fusionDataOutPathGunScream_optAVG_4, fusedModelSavePathGunScream_optAVG_4)


# Evaluating the FUSED model on the training and testing sets

scoreGunshot = OutFusedModelGunScream.evaluate(x_trainGunshot, [y_trainGunshot, np.resize(y_trainScream, (y_trainGunshot.shape[0], *y_trainScream.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Training Accuracy for Gunshot: ", scoreGunshot[3])

scoreGunshot = OutFusedModelGunScream.evaluate(x_testGunshot, [y_testGunshot, np.resize(y_testScream, (y_testGunshot.shape[0], *y_testScream.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Testing Accuracy for Gunshot: ", scoreGunshot[3])



scoreScream= OutFusedModelGunScream.evaluate(x_trainScream, [np.resize(y_trainGunshot, (y_trainScream.shape[0], *y_trainGunshot.shape[1:])), y_trainScream], verbose=0)
print("OUT Scream: ", scoreScream)
print("Training Accuracy for Scream: ", scoreScream[4])

scoreScream= OutFusedModelGunScream.evaluate(x_testScream, [np.resize(y_testGunshot, (y_testScream.shape[0], *y_testGunshot.shape[1:])), y_testScream], verbose=0)
print("OUT Scream: ", scoreScream)
print("Testing Accuracy for Scream: ", scoreScream[4])


y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(OutFusedModelGunScream.predict(x_testGunshot)[0],axis=1)
print('\nConfusion Matrix GUNSHOT:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(OutFusedModelGunScream.predict(x_testScream)[1],axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.


F1-SCORE PROM 1: 
 0.99
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.


F1-SCORE PROM 2: 
 0.99
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases fr

#GUN VS SIREN OPT AVG ARQ 4


In [28]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


fusionDataOutPathGunSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/FusionData/filtersFusion_performance_optAVG_4.xlsx"
fusedModelSavePathGunSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/fused_models/fused_gun_siren_optAVG_4"


OutFusedModelGunSiren = interchange_filters_between_models(modelGunshotTL_4, modelSirenTL_4,  x_testGunshot, y_testGunshot, x_testSiren, y_testSiren, fusionDataOutPathGunSiren_optAVG_4, fusedModelSavePathGunSiren_optAVG_4)


# Evaluating the FUSED model on the training and testing sets

scoreGunshot = OutFusedModelGunSiren.evaluate(x_trainGunshot, [y_trainGunshot, np.resize(y_trainSiren, (y_trainGunshot.shape[0], *y_trainSiren.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Training Accuracy for Gunshot: ", scoreGunshot[3])

scoreGunshot = OutFusedModelGunSiren.evaluate(x_testGunshot, [y_testGunshot, np.resize(y_testSiren, (y_testGunshot.shape[0], *y_testSiren.shape[1:]))], verbose=0)
print("OUT Gunshot: ", scoreGunshot)
print("Testing Accuracy for Gunshot: ", scoreGunshot[3])



scoreSiren= OutFusedModelGunSiren.evaluate(x_trainSiren, [np.resize(y_trainGunshot, (y_trainSiren.shape[0], *y_trainGunshot.shape[1:])), y_trainSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Training Accuracy for Siren: ", scoreSiren[4])

scoreSiren= OutFusedModelGunSiren.evaluate(x_testSiren, [np.resize(y_testGunshot, (y_testSiren.shape[0], *y_testGunshot.shape[1:])), y_testSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Testing Accuracy for Siren: ", scoreSiren[4])


y_true = np.argmax(y_testGunshot,axis=1)
y_pred = np.argmax(OutFusedModelGunSiren.predict(x_testGunshot)[0],axis=1)
print('\nConfusion Matrix GUNSHOT:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(OutFusedModelGunSiren.predict(x_testSiren)[1],axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))


Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.


F1-SCORE PROM 1: 
 0.98
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.


F1-SCORE PROM 2: 
 0.98
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_4' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases 

#SCREAM VS SIREN OPT AVG ARQ 4

In [29]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


fusionDataOutPathScreamSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/FusionData/filtersFusion_performance_optAVG_4.xlsx"
fusedModelSavePathScreamSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/fused_models/fused_scream_siren_optAVG_4"


OutFusedModelScreamSiren = interchange_filters_between_models(modelScreamTL_4, modelSirenTL_4,  x_testScream, y_testScream, x_testSiren, y_testSiren, fusionDataOutPathScreamSiren_optAVG_4, fusedModelSavePathScreamSiren_optAVG_4)


# Evaluating the FUSED model on the training and testing sets

scoreScream= OutFusedModelScreamSiren.evaluate(x_trainScream, [y_trainScream, np.resize(y_trainSiren, (y_trainScream.shape[0], *y_trainSiren.shape[1:]))], verbose=0)
print("OUT SCREAM: ", scoreScream)
print("Training Accuracy for Scream: ", scoreScream[3])

scoreScream = OutFusedModelScreamSiren.evaluate(x_testScream, [y_testScream, np.resize(y_testSiren, (y_testScream.shape[0], *y_testSiren.shape[1:]))], verbose=0)
print("OUT SCREAM: ", scoreScream)
print("Testing Accuracy for Scream: ", scoreScream[3])


scoreSiren= OutFusedModelScreamSiren.evaluate(x_trainSiren, [np.resize(y_trainScream, (y_trainSiren.shape[0], *y_trainScream.shape[1:])), y_trainSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Training Accuracy for Siren: ", scoreSiren[4])

scoreSiren= OutFusedModelScreamSiren.evaluate(x_testSiren, [np.resize(y_testScream, (y_testSiren.shape[0], *y_testScream.shape[1:])), y_testSiren], verbose=0)
print("OUT Siren: ", scoreSiren)
print("Testing Accuracy for Siren: ", scoreSiren[4])


y_true = np.argmax(y_testScream,axis=1)
y_pred = np.argmax(OutFusedModelScreamSiren.predict(x_testScream)[0],axis=1)
print('\nConfusion Matrix SCREAM:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

y_true = np.argmax(y_testSiren,axis=1)
y_pred = np.argmax(OutFusedModelScreamSiren.predict(x_testSiren)[1],axis=1)
print('\nConfusion Matrix SIREN:\n\n')
print('SHAPEEE : ',y_true.shape)
print('SHAPEEE : ',y_pred.shape)
print(confusion_matrix(y_true,y_pred))
print('\n\nClassification Report : \n\n',classification_report(y_true,y_pred))

Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.


F1-SCORE PROM 1: 
 0.98
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.


F1-SCORE PROM 2: 
 0.98
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' and 'conv2d_12' to layer 'conv2d_8' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases from layer 'conv2d_12' to layer 'conv2d_12' one by one.
Transferred filters and biases 

#INTERCAMBIO DE FILTROS BASADO EN EL ARCHIVO DE SIMILARIDAD ENTRE MODELOS  CON PROMEDIO COMO DESEMPATE ARQ 4

In [30]:
import pandas as pd
from scipy.ndimage import rotate

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model


def getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size):
    inputs = Input(shape=(num_rows, num_columns, num_channels))
    x = Conv2D(filters=16, kernel_size=k_size, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=32, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=64, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=128, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)

    output1 = Dense(num_labels, activation='softmax', name='output1')(x)
    output2 = Dense(num_labels, activation='softmax', name='output2')(x)

    model = Model(inputs=inputs, outputs=[output1, output2])
    return model


def get_layer_with_params(model, layer_index):
    layers_with_params = [layer for layer in model.layers if len(layer.trainable_weights) > 0]

    if layer_index < 0 or layer_index >= len(layers_with_params):
        raise ValueError("Invalid layer index.")

    return layers_with_params[layer_index]

def transfer_filters_and_biases_one_by_one(layer_from, layer_to, idx_filter_from, idx_filter_to):
    if len(layer_from.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0:
        return False  # Ambas capas deben tener parámetros entrenables

    filters_from, biases_from = layer_from.get_weights()
    filters_to, biases_to = layer_to.get_weights()

    if filters_from.shape != filters_to.shape or biases_from.shape != biases_to.shape:
        return False  # Las dimensiones de los filtros y sesgos deben coincidir

    filters_to[..., idx_filter_to] = filters_from[..., idx_filter_from]
    #print(filters_to[..., i] - filters_from[..., i])
    biases_to[idx_filter_to] = biases_from[idx_filter_from]  # Copiar el sesgo i de 'layer_from' a 'layer_to'

    layer_to.set_weights([filters_to, biases_to])
    return True

def transfer_filters_and_biases_one_by_one_between_models(model_from, layer_index_from, idx_filter_from, model_to, layer_index_to, idx_filter_to):
    layer_from = get_layer_with_params(model_from, layer_index_from)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_filters_and_biases_one_by_one(layer_from, layer_to, idx_filter_from, idx_filter_to):
        filters_from, biases_from = layer_from.get_weights()
        filters_to, biases_to = layer_to.get_weights()
        #print(f"Transferred filters and biases from layer '{layer_from.name}' to layer '{layer_to.name}' one by one.")
    else:
        print(f"Filters and biases in layers '{layer_from.name}' and '{layer_to.name}' couldn't be transferred one by one.")

    return model_to

def transfer_filters_AVG_and_biases_one_by_one(layer_from1, layer_from2, layer_to, idx_filter_from1, idx_filter_from2, idx_filter_to):
    if len(layer_from1.trainable_weights) == 0 or len(layer_from2.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0:
        return False  # Ambas capas deben tener parámetros entrenables

    filters_from1, biases_from1 = layer_from1.get_weights()
    filters_from2, biases_from2 = layer_from2.get_weights()
    filters_to, biases_to = layer_to.get_weights()

    if filters_from1.shape != filters_to.shape or biases_from1.shape != biases_to.shape or filters_from2.shape != filters_to.shape or biases_from2.shape != biases_to.shape:
        return False  # Las dimensiones de los filtros y sesgos deben coincidir

    # Intercambiar los filtros
    filter_from1 = np.copy(filters_from1[:, :, :, idx_filter_from1])
    filter_from2 = np.copy(filters_from2[:, :, :, idx_filter_from2])

    bias_from1 = np.copy(biases_from1[idx_filter_from1])
    bias_from2 = np.copy(biases_from2[idx_filter_from2])

    # Promediar los pesos y bias de los filtros
    avg_filter = (filter_from1 + filter_from2) / 2
    avg_bias = (bias_from1 + bias_from2) / 2

    filters_to[..., idx_filter_to] = avg_filter
    #print(filters_to[..., i] - avg_filter)
    biases_to[idx_filter_to] = avg_bias # Copiar el sesgo i de 'layer_from' a 'layer_to'

    layer_to.set_weights([filters_to, biases_to])
    return True


def transfer_filters_and_biases_one_by_one_AVG_between_models(model_from1, layer_index_from1, idx_filter_from1, model_from2, layer_index_from2, idx_filter_from2, model_to, layer_index_to, idx_filter_to):
    layer_from1 = get_layer_with_params(model_from1, layer_index_from1)
    layer_from2 = get_layer_with_params(model_from2, layer_index_from2)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_filters_AVG_and_biases_one_by_one(layer_from1, layer_from2, layer_to, idx_filter_from1, idx_filter_from2, idx_filter_to):
        filters_from1, biases_from1 = layer_from1.get_weights()
        filters_from2, biases_from2 = layer_from2.get_weights()
        filters_to, biases_to = layer_to.get_weights()
        #print(f"Transferred filters and biases from layer '{layer_from1.name}' and '{layer_from2.name}' to layer '{layer_to.name}' one by one.")
    else:
        print(f"Filters and biases in layers '{layer_from1.name}' '{layer_from2.name}' and '{layer_to.name}' couldn't be transferred one by one.")

    return model_to



def transfer_layer_weights(layer_from, layer_to):
    if len(layer_from.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0 or ((len(layer_from.trainable_weights)) != (len(layer_to.trainable_weights))):
        return False  # Ambas capas deben tener parámetros entrenables

    weights_from, biases_from = layer_from.get_weights()
    layer_to.set_weights([weights_from, biases_from])
    return True

def transfer_parameters_between_models(model_from, layer_index_from, model_to, layer_index_to):
    layer_from = get_layer_with_params(model_from, layer_index_from)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_layer_weights(layer_from, layer_to):
        print(f"Transferred weights from layer '{layer_from.name}' to layer '{layer_to.name}'.")
    else:
        print(f"Layers '{layer_from.name}' and '{layer_to.name}' don't have trainable parameters.")



def interchange_filters_between_models(model1, model2, x_testModel1, y_testModel1, x_testModel2, y_testModel2, differenceDataPathList, min_similarity, fusionDataOutPath, fusedModelSavePath, rotations=False, reflections=False):

    #1	3	4096	4096	4096	0	2
    #2	12	4096	4096	4096	0	3
    #3	27	4096	4096	4096	0	3
    #4	45	4096	4096	4096	0	5

    samplerate = 22050
    longitudMaxAudio = 4
    Nmfcc = 45
    Nfft = 4096
    NwinL = 4096
    iterableNhopL = 1.0
    NhopL =  4096       #int(iterableNhopL*NwinL)
    k_size = 5
    num_rows = Nmfcc
    num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
    num_channels = 1

    num_labels = 2 #4


    Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

    # Compila el modelo (es necesario antes de la evaluación)
    Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #COPY ORIGINAL MODELS
    model1_copy= keras.models.clone_model(model1)
    model1_copy.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model1_copy.set_weights(model1.get_weights())

    model2_copy= keras.models.clone_model(model2)
    model2_copy.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model2_copy.set_weights(model2.get_weights())

    #TRANFER DENSE LAYERS OF SOFTMAX CLASIFICATION
    transfer_parameters_between_models(model1, 4, Fusedmodel, 4)
    transfer_parameters_between_models(model2, 4, Fusedmodel, 5)


    layer_names = [layer.name for layer in model1.layers if 'conv' in layer.name]

    #INICIALIZO CON EL  PROMEDIO ENTRE AMBOS FILTROS DE LOS MODELOS
    # Update the filters in the models
    for index_layer in range(len(layer_names)):
        layer_name = layer_names[index_layer]
        fusedLayer = get_layer_with_params(Fusedmodel, index_layer)
        #fusedLayer = Fusedmodel.get_layer(layer_name)
        filtersFused, biasesFused = fusedLayer.get_weights()

        layer1 = get_layer_with_params(model1, index_layer)
        #layer1 = model1.get_layer(layer_name)
        filters1, biases1 = layer1.get_weights()

        layer2 = get_layer_with_params(model2, index_layer)
        #layer2 = model2.get_layer(layer_name)
        filters2, biases2 = layer2.get_weights()

        n_filters1 = filters1.shape[-1]
        n_filters2 = filters2.shape[-1]

        # Interchange the filters
        number_filters = 0

        for filter_idx in range(n_filters1):

            model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, model2, index_layer, filter_idx)
            model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, model1, index_layer, filter_idx)

            Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, Fusedmodel, index_layer, filter_idx)

    data = []

    for layer_index in range(len(layer_names)):
        #print("\n\n\n PARA LA CAPA CONV NUMERO: ", layer_index)
        # Read the difference data file
        df = pd.read_excel(differenceDataPathList[layer_index])
        #print(differenceDataPathList[layer_index])
        #print(type(differenceDataPathList[layer_index]))

        #layer_name = layer_names[index_layer]
        #print("\nTAMAÑO ARCHIVO LEIDO")
        #print(df.shape)

        if rotations and reflections:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity)]
        elif rotations:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity) & (df['Reflection'] == 0)]
        elif reflections:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity) & (df['Rotation'] == 0)]
        else:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity) & (df['Reflection'] == 0) & (df['Rotation'] == 0)]

        #return filtered_df
        #print("\nTAMAÑO FILTROS SIMILARES")
        #print(filtered_df.shape)

        # Group the filtered data by filter pairs and find the most similar filters
        #if optimize_model == 1:
            #grouped = filtered_df.groupby('FilterModel1')
            #most_similar = grouped['Similarity'].idxmin()
        #else:
            #grouped = filtered_df.groupby('FilterModel2')
            #most_similar = grouped['Similarity'].idxmin()

        # Interchange the filters
        number_filters = 0
        FiltersIndexChanged = []
        for row in filtered_df.iterrows():
            #print(row[1]["FilterModel1"])

            index_layer = int(row[1]["LayerModel1"])-1
            filter_idx1 = int(row[1]["FilterModel1"])-1
            filter_idx2 = int(row[1]["FilterModel2"])-1

            #if((filter_idx1 in FiltersIndexChanged) or (filter_idx2 in FiltersIndexChanged)):
                #print("Está dentro de la lista cambiado ya alguno de los dos, mirar qué hacer??")
                #if((filter_idx1 in FiltersIndexChanged) and (filter_idx2 in FiltersIndexChanged)):
                    #print("Ambos filtros ya están intercambiados en la lista; no he hecho tratamiento, se puede eliminar redundacia inter modelo?")
            #else:
            if(not((filter_idx1 in FiltersIndexChanged) or (filter_idx2 in FiltersIndexChanged))):
                #En el caso en que no se haya modificado ninguno de los filtros encontrados

                if(filter_idx1 == filter_idx2):
                    #PROBAMOS PRIMERO EL PRIMER FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

                    F1_prom1 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 1: \n', F1_prom1)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #PROBAMOS DESPUES EL SEGUNDO FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


                    F1_prom2 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 2: \n', F1_prom2)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)

                    #VALIDAR CON CUÄL FILTRO SE OBTUVO MEJOR DESEMPEÑO
                    if(F1_prom1>F1_prom2):
                        #ME QUEDO CON EL FILTRO 1 EN AMBOS MODELOS
                        # Update the filters in the models
                        model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                        model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, Fusedmodel, index_layer, filter_idx2)
                        Selected = 'Filter1'
                        FilterChanged = filter_idx2

                    elif(F1_prom1<F1_prom2):
                        #ME QUEDO CON EL FILTRO 2 EN AMBOS MODELOS
                        # Update the filters in the models
                        model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                        model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                        Selected = 'Filter2'
                        FilterChanged = filter_idx1

                    else:
                        #ME QUEDO CON EL  PROMEDIO ENTRE AMBOS FILTROS DE LOS MODELOS
                        # Update the filters in the models
                        model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                        model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                        Selected = 'AVERAGE'
                        FilterChanged = filter_idx1

                    FiltersIndexChanged.append(FilterChanged)
                    data.append((index_layer+1, FilterChanged+1, F1_prom1, F1_prom2, Selected))
                    number_filters += 1


                elif(filter_idx1 != filter_idx2):

                    #PROBAMOS PRIMERO EL PRIMER FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 1 en el indice del filtro 1 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx1)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx1)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

                    F1_prom1 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 1: \n', F1_prom1)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #PROBAMOS DESPUES EL SEGUNDO FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 2 en el indice del filtro 2 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx2)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


                    F1_prom2 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 2: \n', F1_prom2)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #VERSION EN EL INDICE CONTRARIO

                    #PROBAMOS PRIMERO EL PRIMER FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 1 en el indice del filtro 2 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

                    F1_prom12 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 12: \n', F1_prom12)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #PROBAMOS DESPUES EL SEGUNDO FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 2 en el indice del filtro 1 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


                    F1_prom21 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 21: \n', F1_prom21)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #print(F1_prom1, F1_prom2, F1_prom12, F1_prom21)

                    #VALIDAR CON CUÄL FILTRO SE OBTUVO MEJOR DESEMPEÑO
                    if((F1_prom1+F1_prom12)>(F1_prom2+F1_prom21)):
                        #ME QUEDO CON EL FILTRO 1 EN AMBOS MODELOS
                        # Update the filters in the models
                        if(F1_prom12>F1_prom1):
                          #filtro del modelo 1 en el indice del filtro 2 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, Fusedmodel, index_layer, filter_idx2)
                          Selected = 'Filter1'
                          FilterChanged = filter_idx2

                        else:
                          #filtro del modelo 1 en el indice del filtro 1 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx1)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx1)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, Fusedmodel, index_layer, filter_idx1)
                          Selected = 'Filter1'
                          FilterChanged = filter_idx1

                    elif((F1_prom1+F1_prom12)<(F1_prom2+F1_prom21)):
                        #ME QUEDO CON EL FILTRO 2 EN AMBOS MODELOS
                        # Update the filters in the models
                        if(F1_prom21>F1_prom2):
                          #filtro del modelo 2 en el indice del filtro 1 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                          Selected = 'Filter2'
                          FilterChanged = filter_idx1
                        else:
                          #filtro del modelo 2 en el indice del filtro 2 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx2)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx2)
                          Selected = 'Filter2'
                          FilterChanged = filter_idx2

                    else:
                        #ME QUEDO CON EL  PROMEDIO ENTRE AMBOS FILTROS DE LOS MODELOS
                        # Update the filters in the models index1
                        model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                        model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                        Selected = 'AVERAGE'
                        FilterChanged = filter_idx1

                        FiltersIndexChanged.append(FilterChanged)
                        data.append((index_layer+1, FilterChanged+1, F1_prom1, F1_prom2, Selected))
                        number_filters += 1

                        # Update the filters in the models index2
                        model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                        model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx2)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx2)
                        Selected = 'AVERAGE'
                        FilterChanged = filter_idx2

                    FiltersIndexChanged.append(FilterChanged)
                    data.append((index_layer+1, FilterChanged+1, F1_prom1, F1_prom2, Selected))
                    number_filters += 1


    df = pd.DataFrame(data, columns=['LayerModel', 'FilterModel', 'filter1_f1_AVG', 'filter2_f1_AVG', 'Selected'])
    df.to_excel(fusionDataOutPath, index=False)
    print("File saved correctly.")


    print(f"\n\n\n\nFilters interchanged equal to {number_filters}.")
    # Evaluating FINAL PERFORMANCE of models on the training and testing set
    y_true_model2 = np.argmax(y_testModel2,axis=1)
    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
    print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

    y_true_model1 = np.argmax(y_testModel1,axis=1)
    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
    print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


    Fusedmodel.save(fusedModelSavePath)

    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(fusedModelSavePath) # path to the SavedModel directory
    tflite_model = converter.convert()

    # Save the model.
    with open(fusedModelSavePath+".tflite", 'wb') as f:
      f.write(tflite_model)

    print("Saved Models Correctly.")

    return Fusedmodel


#GUN VS SCREAM OPT AVG ARQ 4

In [55]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


differenceDataPathL0 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric1/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer0_4.xlsx"
differenceDataPathL1 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric1/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer1_4.xlsx"
differenceDataPathL2 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric1/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer2_4.xlsx"
differenceDataPathL3 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric1/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer3_4.xlsx"

differenceDataPathList = [differenceDataPathL0, differenceDataPathL1, differenceDataPathL2, differenceDataPathL3]


fusionDataOutPathGunScream_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/FusionData/filtersFusion_sim20_optAVG_4.xlsx"
fusedModelSavePathGunScream_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/fused_models/fused_gun_scream_sim20_optAVG_4"

min_similarity = 0.20  # Specify the min similarity fusion treashold (e.g., 0.9 for 90% of similarity)

df = interchange_filters_between_models(modelGunshotTL_4, modelScreamTL_4,  x_testGunshot, y_testGunshot, x_testScream, y_testScream, differenceDataPathList, min_similarity, fusionDataOutPathGunScream_optAVG_4, fusedModelSavePathGunScream_optAVG_4, rotations=False, reflections=False)


Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
File saved correctly.




Filters interchanged equal to 128.


Classification Report SCREAM  - MODEL2 F1-SCORE: 

 0.97


Classification Report GUN - MODEL1 F1-SCORE: 

 0.99
Saved Models Correctly.


#GUN VS SIREN OPT AVG ARQ 4

In [56]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


differenceDataPathL0 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric1/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer0_4.xlsx"
differenceDataPathL1 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric1/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer1_4.xlsx"
differenceDataPathL2 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric1/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer2_4.xlsx"
differenceDataPathL3 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric1/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer3_4.xlsx"

differenceDataPathList = [differenceDataPathL0, differenceDataPathL1, differenceDataPathL2, differenceDataPathL3]


fusionDataOutPathGunSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/FusionData/filtersFusion_sim20_optAVG_4.xlsx"
fusedModelSavePathGunSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/fused_models/fused_gun_siren_sim20_optAVG_4"

min_similarity = 0.20  # Specify the min similarity fusion treashold (e.g., 0.9 for 90% of similarity)

df = interchange_filters_between_models(modelGunshotTL_4, modelSirenTL_4,  x_testGunshot, y_testGunshot, x_testSiren, y_testSiren, differenceDataPathList, min_similarity, fusionDataOutPathGunSiren_optAVG_4, fusedModelSavePathGunSiren_optAVG_4, rotations=False, reflections=False)


Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
File saved correctly.




Filters interchanged equal to 128.


Classification Report SCREAM  - MODEL2 F1-SCORE: 

 0.96


Classification Report GUN - MODEL1 F1-SCORE: 

 0.97
Saved Models Correctly.


#SCREAM VS SIREN OPT AVG ARQ 4

In [57]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


differenceDataPathL0 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric1/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer0_4.xlsx"
differenceDataPathL1 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric1/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer1_4.xlsx"
differenceDataPathL2 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric1/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer2_4.xlsx"
differenceDataPathL3 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric1/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer3_4.xlsx"

differenceDataPathList = [differenceDataPathL0, differenceDataPathL1, differenceDataPathL2, differenceDataPathL3]


fusionDataOutPathScreamSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/FusionData/filtersFusion_sim20_optAVG_4.xlsx"
fusedModelSavePathScreamSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/fused_models/fused_scream_siren_sim20_optAVG_4"

min_similarity = 0.20  # Specify the min similarity fusion treashold (e.g., 0.9 for 90% of similarity)

df = interchange_filters_between_models(modelScreamTL_4, modelSirenTL_4,  x_testScream, y_testScream, x_testSiren, y_testSiren, differenceDataPathList, min_similarity, fusionDataOutPathScreamSiren_optAVG_4, fusedModelSavePathScreamSiren_optAVG_4, rotations=False, reflections=False)


Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
File saved correctly.




Filters interchanged equal to 128.


Classification Report SCREAM  - MODEL2 F1-SCORE: 

 0.97


Classification Report GUN - MODEL1 F1-SCORE: 

 0.99
Saved Models Correctly.


#INTERCAMBIO DE FILTROS BASADO EN EL ARCHIVO DE SIMILARIDAD NORMALIZADO ENTRE MODELOS  CON PROMEDIO COMO DESEMPATE ARQ 4

In [26]:
import pandas as pd
from scipy.ndimage import rotate

from tensorflow.keras.layers import Input, Conv2D, MaxPooling2D, Dropout, GlobalAveragePooling2D, Flatten, Dense
from tensorflow.keras.models import Model


def getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size):
    inputs = Input(shape=(num_rows, num_columns, num_channels))
    x = Conv2D(filters=16, kernel_size=k_size, activation='relu', padding='same')(inputs)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=32, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=64, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = Conv2D(filters=128, kernel_size=k_size, activation='relu', padding='same')(x)
    x = MaxPooling2D(pool_size=(2,2), padding='same')(x)
    x = Dropout(0.2)(x)

    x = GlobalAveragePooling2D()(x)
    x = Flatten()(x)

    output1 = Dense(num_labels, activation='softmax', name='output1')(x)
    output2 = Dense(num_labels, activation='softmax', name='output2')(x)

    model = Model(inputs=inputs, outputs=[output1, output2])
    return model


def get_layer_with_params(model, layer_index):
    layers_with_params = [layer for layer in model.layers if len(layer.trainable_weights) > 0]

    if layer_index < 0 or layer_index >= len(layers_with_params):
        raise ValueError("Invalid layer index.")

    return layers_with_params[layer_index]

def transfer_filters_and_biases_one_by_one(layer_from, layer_to, idx_filter_from, idx_filter_to):
    if len(layer_from.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0:
        return False  # Ambas capas deben tener parámetros entrenables

    filters_from, biases_from = layer_from.get_weights()
    filters_to, biases_to = layer_to.get_weights()

    if filters_from.shape != filters_to.shape or biases_from.shape != biases_to.shape:
        return False  # Las dimensiones de los filtros y sesgos deben coincidir

    filters_to[..., idx_filter_to] = filters_from[..., idx_filter_from]
    #print(filters_to[..., i] - filters_from[..., i])
    biases_to[idx_filter_to] = biases_from[idx_filter_from]  # Copiar el sesgo i de 'layer_from' a 'layer_to'

    layer_to.set_weights([filters_to, biases_to])
    return True

def transfer_filters_and_biases_one_by_one_between_models(model_from, layer_index_from, idx_filter_from, model_to, layer_index_to, idx_filter_to):
    layer_from = get_layer_with_params(model_from, layer_index_from)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_filters_and_biases_one_by_one(layer_from, layer_to, idx_filter_from, idx_filter_to):
        filters_from, biases_from = layer_from.get_weights()
        filters_to, biases_to = layer_to.get_weights()
        #print(f"Transferred filters and biases from layer '{layer_from.name}' to layer '{layer_to.name}' one by one.")
    else:
        print(f"Filters and biases in layers '{layer_from.name}' and '{layer_to.name}' couldn't be transferred one by one.")

    return model_to

def transfer_filters_AVG_and_biases_one_by_one(layer_from1, layer_from2, layer_to, idx_filter_from1, idx_filter_from2, idx_filter_to):
    if len(layer_from1.trainable_weights) == 0 or len(layer_from2.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0:
        return False  # Ambas capas deben tener parámetros entrenables

    filters_from1, biases_from1 = layer_from1.get_weights()
    filters_from2, biases_from2 = layer_from2.get_weights()
    filters_to, biases_to = layer_to.get_weights()

    if filters_from1.shape != filters_to.shape or biases_from1.shape != biases_to.shape or filters_from2.shape != filters_to.shape or biases_from2.shape != biases_to.shape:
        return False  # Las dimensiones de los filtros y sesgos deben coincidir

    # Intercambiar los filtros
    filter_from1 = np.copy(filters_from1[:, :, :, idx_filter_from1])
    filter_from2 = np.copy(filters_from2[:, :, :, idx_filter_from2])

    bias_from1 = np.copy(biases_from1[idx_filter_from1])
    bias_from2 = np.copy(biases_from2[idx_filter_from2])

    # Promediar los pesos y bias de los filtros
    avg_filter = (filter_from1 + filter_from2) / 2
    avg_bias = (bias_from1 + bias_from2) / 2

    filters_to[..., idx_filter_to] = avg_filter
    #print(filters_to[..., i] - avg_filter)
    biases_to[idx_filter_to] = avg_bias # Copiar el sesgo i de 'layer_from' a 'layer_to'

    layer_to.set_weights([filters_to, biases_to])
    return True


def transfer_filters_and_biases_one_by_one_AVG_between_models(model_from1, layer_index_from1, idx_filter_from1, model_from2, layer_index_from2, idx_filter_from2, model_to, layer_index_to, idx_filter_to):
    layer_from1 = get_layer_with_params(model_from1, layer_index_from1)
    layer_from2 = get_layer_with_params(model_from2, layer_index_from2)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_filters_AVG_and_biases_one_by_one(layer_from1, layer_from2, layer_to, idx_filter_from1, idx_filter_from2, idx_filter_to):
        filters_from1, biases_from1 = layer_from1.get_weights()
        filters_from2, biases_from2 = layer_from2.get_weights()
        filters_to, biases_to = layer_to.get_weights()
        #print(f"Transferred filters and biases from layer '{layer_from1.name}' and '{layer_from2.name}' to layer '{layer_to.name}' one by one.")
    else:
        print(f"Filters and biases in layers '{layer_from1.name}' '{layer_from2.name}' and '{layer_to.name}' couldn't be transferred one by one.")

    return model_to



def transfer_layer_weights(layer_from, layer_to):
    if len(layer_from.trainable_weights) == 0 or len(layer_to.trainable_weights) == 0 or ((len(layer_from.trainable_weights)) != (len(layer_to.trainable_weights))):
        return False  # Ambas capas deben tener parámetros entrenables

    weights_from, biases_from = layer_from.get_weights()
    layer_to.set_weights([weights_from, biases_from])
    return True

def transfer_parameters_between_models(model_from, layer_index_from, model_to, layer_index_to):
    layer_from = get_layer_with_params(model_from, layer_index_from)
    layer_to = get_layer_with_params(model_to, layer_index_to)

    if transfer_layer_weights(layer_from, layer_to):
        print(f"Transferred weights from layer '{layer_from.name}' to layer '{layer_to.name}'.")
    else:
        print(f"Layers '{layer_from.name}' and '{layer_to.name}' don't have trainable parameters.")



def interchange_filters_between_models(model1, model2, x_testModel1, y_testModel1, x_testModel2, y_testModel2, differenceDataPathList, min_similarity, fusionDataOutPath, fusedModelSavePath, rotations=False, reflections=False):

    #1	3	4096	4096	4096	0	2
    #2	12	4096	4096	4096	0	3
    #3	27	4096	4096	4096	0	3
    #4	45	4096	4096	4096	0	5

    samplerate = 22050
    longitudMaxAudio = 4
    Nmfcc = 45
    Nfft = 4096
    NwinL = 4096
    iterableNhopL = 1.0
    NhopL =  4096       #int(iterableNhopL*NwinL)
    k_size = 5
    num_rows = Nmfcc
    num_columns = int(samplerate*longitudMaxAudio/NhopL) + int(samplerate*longitudMaxAudio/NhopL*0.05)  #Calculo longitud de salida de mfcc con 5% de tolerancia para longitud de audios
    num_channels = 1

    num_labels = 2 #4


    Fusedmodel = getModelFunctionalModified(num_rows, num_columns, num_channels, num_labels, k_size)

    # Compila el modelo (es necesario antes de la evaluación)
    Fusedmodel.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

    #COPY ORIGINAL MODELS
    model1_copy= keras.models.clone_model(model1)
    model1_copy.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model1_copy.set_weights(model1.get_weights())

    model2_copy= keras.models.clone_model(model2)
    model2_copy.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    model2_copy.set_weights(model2.get_weights())

    #TRANFER DENSE LAYERS OF SOFTMAX CLASIFICATION
    transfer_parameters_between_models(model1, 4, Fusedmodel, 4)
    transfer_parameters_between_models(model2, 4, Fusedmodel, 5)


    layer_names = [layer.name for layer in model1.layers if 'conv' in layer.name]

    #INICIALIZO CON EL  PROMEDIO ENTRE AMBOS FILTROS DE LOS MODELOS
    # Update the filters in the models
    for index_layer in range(len(layer_names)):
        layer_name = layer_names[index_layer]
        fusedLayer = get_layer_with_params(Fusedmodel, index_layer)
        #fusedLayer = Fusedmodel.get_layer(layer_name)
        filtersFused, biasesFused = fusedLayer.get_weights()

        layer1 = get_layer_with_params(model1, index_layer)
        #layer1 = model1.get_layer(layer_name)
        filters1, biases1 = layer1.get_weights()

        layer2 = get_layer_with_params(model2, index_layer)
        #layer2 = model2.get_layer(layer_name)
        filters2, biases2 = layer2.get_weights()

        n_filters1 = filters1.shape[-1]
        n_filters2 = filters2.shape[-1]

        # Interchange the filters
        number_filters = 0

        for filter_idx in range(n_filters1):

            model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, model2, index_layer, filter_idx)
            model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, model1, index_layer, filter_idx)

            Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx, model2_copy, index_layer, filter_idx, Fusedmodel, index_layer, filter_idx)

    data = []

    for layer_index in range(len(layer_names)):
        #print("\n\n\n PARA LA CAPA CONV NUMERO: ", layer_index)
        # Read the difference data file
        df = pd.read_excel(differenceDataPathList[layer_index])
        #print(differenceDataPathList[layer_index])
        #print(type(differenceDataPathList[layer_index]))

        #layer_name = layer_names[index_layer]
        #print("\nTAMAÑO ARCHIVO LEIDO")
        #print(df.shape)

        if rotations and reflections:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity)]
        elif rotations:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity) & (df['Reflection'] == 0)]
        elif reflections:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity) & (df['Rotation'] == 0)]
        else:
          # Filter the data for the specified layer index and minimum similarity
          filtered_df = df[(df['LayerModel1'] == layer_index+1) & (df['LayerModel2'] == layer_index+1) & (df['Similarity'] >= min_similarity) & (df['Reflection'] == 0) & (df['Rotation'] == 0)]

        #return filtered_df
        #print("\nTAMAÑO FILTROS SIMILARES")
        #print(filtered_df.shape)

        # Group the filtered data by filter pairs and find the most similar filters
        #if optimize_model == 1:
            #grouped = filtered_df.groupby('FilterModel1')
            #most_similar = grouped['Similarity'].idxmin()
        #else:
            #grouped = filtered_df.groupby('FilterModel2')
            #most_similar = grouped['Similarity'].idxmin()

        # Interchange the filters
        number_filters = 0
        FiltersIndexChanged = []
        for row in filtered_df.iterrows():
            #print(row[1]["FilterModel1"])

            index_layer = int(row[1]["LayerModel1"])-1
            filter_idx1 = int(row[1]["FilterModel1"])-1
            filter_idx2 = int(row[1]["FilterModel2"])-1

            #if((filter_idx1 in FiltersIndexChanged) or (filter_idx2 in FiltersIndexChanged)):
                #print("Está dentro de la lista cambiado ya alguno de los dos, mirar qué hacer??")
                #if((filter_idx1 in FiltersIndexChanged) and (filter_idx2 in FiltersIndexChanged)):
                    #print("Ambos filtros ya están intercambiados en la lista; no he hecho tratamiento, se puede eliminar redundacia inter modelo?")
            #else:
            if(not((filter_idx1 in FiltersIndexChanged) or (filter_idx2 in FiltersIndexChanged))):
                #En el caso en que no se haya modificado ninguno de los filtros encontrados

                if(filter_idx1 == filter_idx2):
                    #PROBAMOS PRIMERO EL PRIMER FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

                    F1_prom1 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 1: \n', F1_prom1)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #PROBAMOS DESPUES EL SEGUNDO FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


                    F1_prom2 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 2: \n', F1_prom2)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)

                    #VALIDAR CON CUÄL FILTRO SE OBTUVO MEJOR DESEMPEÑO
                    if(F1_prom1>F1_prom2):
                        #ME QUEDO CON EL FILTRO 1 EN AMBOS MODELOS
                        # Update the filters in the models
                        model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                        model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, Fusedmodel, index_layer, filter_idx2)
                        Selected = 'Filter1'
                        FilterChanged = filter_idx2

                    elif(F1_prom1<F1_prom2):
                        #ME QUEDO CON EL FILTRO 2 EN AMBOS MODELOS
                        # Update the filters in the models
                        model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                        model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                        Selected = 'Filter2'
                        FilterChanged = filter_idx1

                    else:
                        #ME QUEDO CON EL  PROMEDIO ENTRE AMBOS FILTROS DE LOS MODELOS
                        # Update the filters in the models
                        model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                        model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                        Selected = 'AVERAGE'
                        FilterChanged = filter_idx1

                    FiltersIndexChanged.append(FilterChanged)
                    data.append((index_layer+1, FilterChanged+1, F1_prom1, F1_prom2, Selected))
                    number_filters += 1


                elif(filter_idx1 != filter_idx2):

                    #PROBAMOS PRIMERO EL PRIMER FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 1 en el indice del filtro 1 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx1)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx1)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

                    F1_prom1 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 1: \n', F1_prom1)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #PROBAMOS DESPUES EL SEGUNDO FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 2 en el indice del filtro 2 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx2)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


                    F1_prom2 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 2: \n', F1_prom2)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #VERSION EN EL INDICE CONTRARIO

                    #PROBAMOS PRIMERO EL PRIMER FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 1 en el indice del filtro 2 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])

                    F1_prom12 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 12: \n', F1_prom12)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #PROBAMOS DESPUES EL SEGUNDO FILTRO EN AMBOS MODELOS Y EVALUAMOS DESEMPEÑO
                    #filtro del modelo 2 en el indice del filtro 1 en ambos modelos
                    model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                    model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                    # Evaluating models on the training and testing set
                    y_true_model2 = np.argmax(y_testModel2,axis=1)
                    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
                    #print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
                    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

                    y_true_model1 = np.argmax(y_testModel1,axis=1)
                    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
                    #print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
                    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


                    F1_prom21 = (f1_score_model2+f1_score_model1)/2
                    #print('\n\nF1-SCORE PROM 21: \n', F1_prom21)
                    #print('\nF1-SCORE MODEL 1: ', f1_score_model1)
                    #print('\nF1-SCORE MODEL 2: ', f1_score_model2)


                    #print(F1_prom1, F1_prom2, F1_prom12, F1_prom21)

                    #VALIDAR CON CUÄL FILTRO SE OBTUVO MEJOR DESEMPEÑO
                    if((F1_prom1+F1_prom12)>(F1_prom2+F1_prom21)):
                        #ME QUEDO CON EL FILTRO 1 EN AMBOS MODELOS
                        # Update the filters in the models
                        if(F1_prom12>F1_prom1):
                          #filtro del modelo 1 en el indice del filtro 2 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx2)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx2)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, Fusedmodel, index_layer, filter_idx2)
                          Selected = 'Filter1'
                          FilterChanged = filter_idx2

                        else:
                          #filtro del modelo 1 en el indice del filtro 1 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model2, index_layer, filter_idx1)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, model1, index_layer, filter_idx1)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model1_copy, index_layer, filter_idx1, Fusedmodel, index_layer, filter_idx1)
                          Selected = 'Filter1'
                          FilterChanged = filter_idx1

                    elif((F1_prom1+F1_prom12)<(F1_prom2+F1_prom21)):
                        #ME QUEDO CON EL FILTRO 2 EN AMBOS MODELOS
                        # Update the filters in the models
                        if(F1_prom21>F1_prom2):
                          #filtro del modelo 2 en el indice del filtro 1 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                          Selected = 'Filter2'
                          FilterChanged = filter_idx1
                        else:
                          #filtro del modelo 2 en el indice del filtro 2 en ambos modelos
                          model2 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                          model1 = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx2)

                          Fusedmodel = transfer_filters_and_biases_one_by_one_between_models(model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx2)
                          Selected = 'Filter2'
                          FilterChanged = filter_idx2

                    else:
                        #ME QUEDO CON EL  PROMEDIO ENTRE AMBOS FILTROS DE LOS MODELOS
                        # Update the filters in the models index1
                        model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx1)
                        model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx1)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx1)
                        Selected = 'AVERAGE'
                        FilterChanged = filter_idx1

                        FiltersIndexChanged.append(FilterChanged)
                        data.append((index_layer+1, FilterChanged+1, F1_prom1, F1_prom2, Selected))
                        number_filters += 1

                        # Update the filters in the models index2
                        model2 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model2, index_layer, filter_idx2)
                        model1 = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, model1, index_layer, filter_idx2)

                        Fusedmodel = transfer_filters_and_biases_one_by_one_AVG_between_models(model1_copy, index_layer, filter_idx1, model2_copy, index_layer, filter_idx2, Fusedmodel, index_layer, filter_idx2)
                        Selected = 'AVERAGE'
                        FilterChanged = filter_idx2

                    FiltersIndexChanged.append(FilterChanged)
                    data.append((index_layer+1, FilterChanged+1, F1_prom1, F1_prom2, Selected))
                    number_filters += 1


    df = pd.DataFrame(data, columns=['LayerModel', 'FilterModel', 'filter1_f1_AVG', 'filter2_f1_AVG', 'Selected'])
    df.to_excel(fusionDataOutPath, index=False)
    print("File saved correctly.")


    print(f"\n\n\n\nFilters interchanged equal to {number_filters}.")
    # Evaluating FINAL PERFORMANCE of models on the training and testing set
    y_true_model2 = np.argmax(y_testModel2,axis=1)
    y_pred_model2 = np.argmax(model2.predict(x_testModel2, verbose=0),axis=1)
    print('\n\nClassification Report SCREAM  - MODEL2 F1-SCORE: \n\n', float(classification_report(y_true_model2,y_pred_model2).split()[15]))
    f1_score_model2 = float(classification_report(y_true_model2,y_pred_model2).split()[15])

    y_true_model1 = np.argmax(y_testModel1,axis=1)
    y_pred_model1 = np.argmax(model1.predict(x_testModel1, verbose=0),axis=1)
    print('\n\nClassification Report GUN - MODEL1 F1-SCORE: \n\n', float(classification_report(y_true_model1,y_pred_model1).split()[15]))
    f1_score_model1 = float(classification_report(y_true_model1,y_pred_model1).split()[15])


    Fusedmodel.save(fusedModelSavePath)

    # Convert the model
    converter = tf.lite.TFLiteConverter.from_saved_model(fusedModelSavePath) # path to the SavedModel directory
    tflite_model = converter.convert()

    # Save the model.
    with open(fusedModelSavePath+".tflite", 'wb') as f:
      f.write(tflite_model)

    print("Saved Models Correctly.")

    return Fusedmodel


#GUN VS SCREAM OPT AVG ARQ 4 NORM

In [53]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


differenceDataPathL0 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric2/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer0_4_N.xlsx"
differenceDataPathL1 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric2/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer1_4_N.xlsx"
differenceDataPathL2 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric2/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer2_4_N.xlsx"
differenceDataPathL3 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/Comparison/Metric2/gun_TL_vs_scream_TL/gunshotTL_vs_screamTL_Layer3_4_N.xlsx"


differenceDataPathList = [differenceDataPathL0, differenceDataPathL1, differenceDataPathL2, differenceDataPathL3]


fusionDataOutPathGunScream_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/FusionData/filtersFusion_sim20_optAVG_4_N.xlsx"
fusedModelSavePathGunScream_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_scream/fused_models/fused_gun_scream_sim20_optAVG_4_N"

min_similarity = 0.20  # Specify the min similarity fusion treashold (e.g., 0.9 for 90% of similarity)

df = interchange_filters_between_models(modelGunshotTL_4, modelScreamTL_4,  x_testGunshot, y_testGunshot, x_testScream, y_testScream, differenceDataPathList, min_similarity, fusionDataOutPathGunScream_optAVG_4, fusedModelSavePathGunScream_optAVG_4, rotations=False, reflections=False)


Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
File saved correctly.




Filters interchanged equal to 128.


Classification Report SCREAM  - MODEL2 F1-SCORE: 

 0.97


Classification Report GUN - MODEL1 F1-SCORE: 

 0.98
Saved Models Correctly.


#GUN VS SIREN OPT AVG ARQ 4 NORM

In [52]:
mmodelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


differenceDataPathL0 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric2/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer0_4_N.xlsx"
differenceDataPathL1 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric2/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer1_4_N.xlsx"
differenceDataPathL2 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric2/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer2_4_N.xlsx"
differenceDataPathL3 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/Comparison/Metric2/gun_TL_vs_siren_TL/gunshotTL_vs_sirenTL_Layer3_4_N.xlsx"

differenceDataPathList = [differenceDataPathL0, differenceDataPathL1, differenceDataPathL2, differenceDataPathL3]


fusionDataOutPathGunSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/FusionData/filtersFusion_sim20_optAVG_4_N.xlsx"
fusedModelSavePathGunSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/gun_siren/fused_models/fused_gun_siren_sim20_optAVG_4_N"

min_similarity = 0.20  # Specify the min similarity fusion treashold (e.g., 0.9 for 90% of similarity)

df = interchange_filters_between_models(modelGunshotTL_4, modelSirenTL_4,  x_testGunshot, y_testGunshot, x_testSiren, y_testSiren, differenceDataPathList, min_similarity, fusionDataOutPathGunSiren_optAVG_4, fusedModelSavePathGunSiren_optAVG_4, rotations=False, reflections=False)


Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
File saved correctly.




Filters interchanged equal to 128.


Classification Report SCREAM  - MODEL2 F1-SCORE: 

 0.94


Classification Report GUN - MODEL1 F1-SCORE: 

 0.98
Saved Models Correctly.


#SCREAM VS SIREN OPT AVG ARQ 4 NORM

In [51]:
modelGunshotTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/gunshot/saved_gunshot_TL_4'
modelGunshotTL_4 = tf.keras.models.load_model(modelGunshotTLSave_path_4)

modelScreamTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/scream/saved_scream_TL_4'
modelScreamTL_4 = tf.keras.models.load_model(modelScreamTLSave_path_4)

modelSirenTLSave_path_4 = '/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/TL_Complete/siren/saved_siren_TL_4'
modelSirenTL_4 = tf.keras.models.load_model(modelSirenTLSave_path_4)


differenceDataPathL0 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric2/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer0_4_N.xlsx"
differenceDataPathL1 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric2/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer1_4_N.xlsx"
differenceDataPathL2 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric2/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer2_4_N.xlsx"
differenceDataPathL3 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/Comparison/Metric2/scream_TL_vs_siren_TL/screamTL_vs_sirenTL_Layer3_4_N.xlsx"

differenceDataPathList = [differenceDataPathL0, differenceDataPathL1, differenceDataPathL2, differenceDataPathL3]


fusionDataOutPathScreamSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/FusionData/filtersFusion_sim20_optAVG_4_N.xlsx"
fusedModelSavePathScreamSiren_optAVG_4 =  "/content/drive/MyDrive/Colab Notebooks/Urban Sound Detection/Models/Fusion/scream_siren/fused_models/fused_scream_siren_sim20_optAVG_4_N"

min_similarity = 0.20  # Specify the min similarity fusion treashold (e.g., 0.9 for 90% of similarity)

df = interchange_filters_between_models(modelScreamTL_4, modelSirenTL_4,  x_testScream, y_testScream, x_testSiren, y_testSiren, differenceDataPathList, min_similarity, fusionDataOutPathScreamSiren_optAVG_4, fusedModelSavePathScreamSiren_optAVG_4, rotations=False, reflections=False)


Transferred weights from layer 'dense_3' to layer 'output1'.
Transferred weights from layer 'dense_3' to layer 'output2'.
File saved correctly.




Filters interchanged equal to 128.


Classification Report SCREAM  - MODEL2 F1-SCORE: 

 0.96


Classification Report GUN - MODEL1 F1-SCORE: 

 0.99
Saved Models Correctly.
